# 3.- Azure ML Resources

In [13]:
import yaml

In [14]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

## Create Resource Group

In [7]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

In [ ]:
# Credentials and params
subscription_id = config["azure"]["subscription_id"]
credential = DefaultAzureCredential()
resource_client = ResourceManagementClient(credential, subscription_id)

# Create resource group
resource_group_name = "satellite_object_detection_resource_group"
location = "eastus"

resource_client.resource_groups.create_or_update(
    resource_group_name,
    {"location": location}
)
print(f"Resource Group '{resource_group_name}' created in {location}")

## Create Workspace

In [2]:
from azureml.core import Workspace

In [ ]:
workspace_name = "satellite_object_detection_workspace_ml"

# Create Workspace
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group_name,
    location=location,
    exist_ok=True
)
print(f"Workspace '{workspace_name}' created or retrieved successfully")

## Create Compute Cluster

In [3]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException

In [ ]:
compute_name = "gpu-cluster"
vm_size = "Standard_NC6"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print(f"The cluster '{compute_name}' already exists")
except ComputeTargetException:
    print(f"Creating the cluster '{compute_name}'...")
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

## Upload Data to Datastore

In [16]:
from azureml.core import Datastore

In [18]:
local_data_path = '../data'

In [ ]:
datastore = ws.get_default_datastore()

datastore.upload(src_dir=local_data_path, target_path="object-detection-data", overwrite=True)
print("Data successfully uploaded to the datastore")